In [19]:
%load_ext autoreload  
%autoreload 2  
  
import random  
import numpy as np  
import torch  
import torch.nn as nn  
from braindecode import EEGClassifier

# dataset related  
from modules.competition_dataset import EEGDataset
from torch.utils.data import TensorDataset, DataLoader
from braindecode.models import EEGSimpleConv, EEGInceptionERP
from skorch.helper import predefined_split  
import random
import mne
import numpy as np
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
device  

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


device(type='cpu')

In [38]:
data_path = './data/mtcaic3'
model_path = './checkpoints/ssvep/models/the_honored_one.pth'
optuna_db_path = './checkpoints/ssvep/optuna/the_honored_one.db'
eeg_channels = [
    "OZ", 
    "PO7",
    "PO8",
    "PZ",
]


batch_size = 64
# Add this at the beginning of your notebook, after imports
def set_random_seeds(seed=42):
    """Set random seeds for reproducibility"""

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# Call this function before creating datasets and models
set_random_seeds(42)

In [43]:
window_length = 128 * 3 # ensure divisble by 64 the kernel size
stride = 20
batch_size = 64

dataset_train = EEGDataset(
    data_path,
    window_length=window_length,
    stride=stride,
    data_fraction=0.2,
    task="SSVEP",
    eeg_channels=eeg_channels,
    tmin=1,
)

dataset_val = EEGDataset(
    data_path=data_path,
    window_length=window_length,
    stride=stride,
    split='validation',
    data_fraction=1,
    task="SSVEP",
    eeg_channels=eeg_channels,
    tmin=1,
)


# train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
# val_loader   = DataLoader(dataset_val,   batch_size=batch_size)

task: SSVEP, split: train, domain: time, data_fraction: 0.2
Using 20.0% of data: 480/480 samples
skipped: 1/480
task: SSVEP, split: validation, domain: time, data_fraction: 1
skipped: 0/50


In [44]:
print(dataset_val.labels.shape)  # should be (N,)
print(dataset_val.labels[:10])   # should be 0, 1, 2, 3

torch.Size([3196])
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])


In [ ]:
model = EEGSimpleConv(
    n_chans=4,
    n_outputs=4,  # Left/right/backward/forward
    sfreq=80,  # Optimal resampling frequency for this model is 80hz
    feature_maps=96,  # Within recommended range [64-144]
    n_convs=2,  # For cross-subject: [2-4]
    kernel_size=8,  # For cross-subject: [5-8]
    resampling_freq=80,
)

clf = EEGClassifier(
    model,
    criterion=torch.nn.CrossEntropyLoss,
    optimizer=torch.optim.Adam,
    optimizer__lr=0.0001,
    batch_size=64,
    max_epochs=100,
    train_split=predefined_split(dataset_val),
    device="cuda" if torch.cuda.is_available() else "cpu",
    verbose=2,
    callbacks=["accuracy"],
)

clf.fit(dataset_train.data, dataset_train.labels)

  epoch    train_accuracy    train_loss    valid_acc    valid_accuracy    valid_loss      dur
-------  ----------------  ------------  -----------  ----------------  ------------  -------
      1            0.4553        1.2680       0.2797            0.2797        3.3358  83.6039
      2            0.4605        1.2040       0.1599            0.1599        3.3829  87.3383
      3            0.5018        1.1706       0.1974            0.1974        4.3919  94.5323
      4            0.5326        1.1427       0.1934            0.1934        3.4617  81.6469
      5            0.5058        1.1246       0.2365            0.2365        3.8485  82.9627
      6            0.4997        1.1064       0.2031            0.2031        3.1055  84.2994
      7            0.5187        1.0892       0.2569            0.2569        4.5766  77.1784
      8            0.5296        1.0722       0.2547            0.2547        4.1038  92.9631
      9            0.5247        1.0551       0.2722        

In [ ]:
print(X_train.shape, y_train.shape)

model = ATCNet(
    n_chans=5,
    n_outputs=2,  # Left/right hand motor imagery
    input_window_seconds=3.0,
    sfreq=250,
    n_windows=5,
    att_head_dim=8,
    att_num_heads=2,
    tcn_depth=2,
    tcn_kernel_size=4,
    tcn_n_filters=32,
)

clf = EEGClassifier(
    model,
    criterion=torch.nn.CrossEntropyLoss,
    optimizer=torch.optim.Adam,
    optimizer__lr=0.0005,  # Lower learning rate for attention models
    batch_size=32,  # Smaller batch size for memory efficiency
    max_epochs=100,
    train_split=predefined_split(val_dataset),
    device="cuda" if torch.cuda.is_available() else "cpu",
    verbose=2,
    callbacks=["accuracy"],
)

clf.fit(X_train, y_train)